In [ ]:
import pandas as pd
from numpy.random import randint, randn, shuffle
from numpy import expand_dims, zeros, ones, vstack
import numpy as np
from matplotlib import pyplot
%matplotlib inline

from keras.models import Sequential
from keras.layers import Conv2D, Conv2DTranspose, Dropout, Flatten, LeakyReLU, Flatten, Reshape, Dense
from keras.optimizers import Adam

In [ ]:
def discriminator(input_shape=(28,28,1)):
    model = Sequential()
        
    model.add(Conv2D(128, kernel_size=(2,2), strides=(2,2), input_shape=input_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
        
    model.add(Conv2D(64, kernel_size=(3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
        
    model.add(Conv2D(32, kernel_size=(5,5), strides=(1,1),padding='same'))
    model.add(Dropout(0.4))
    model.add(Flatten())
        
    model.add(Dense(64, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='relu'))
        
    model.compile(optimizer='Adam', loss='binary_crossentropy',  metrics=['accuracy'])
    return model

In [ ]:
def generator(latent_dim):
    model = Sequential()
        
    n_nodes = 128*7*7
    # start with 7x7 image
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
        
    # upsample to 14x14
    model.add(Conv2DTranspose(128, (2,2), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
        
    # usample to 28x28
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))          
    model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
        
    model.compile(optimizer='Adam', loss='binary_crossentropy')
                  
    return model

In [ ]:
def GAN(gen, dis):
    dis.trainable = False
    model = Sequential()
    model.add(gen)
    model.add(dis)
        
    model.compile(optimizer='Adam', loss='binary_crossentropy')
        
    return model
    
def latent_point(latent_dim, n_sample):
    x_input = randn(latent_dim * n_sample)
    x_input = x_input.reshape(n_sample, latent_dim)
    return x_input
    
def real_sample(n_sample, data):
    rand_ind = randint(0, data.shape[0], n_sample)
    x = data[rand_ind]
    x = x.reshape(-1,28,28,1)
    y = ones((n_sample,1))
        
    return x, y
    
def fake_sample(gen, n_sample, latent_dim=100):
    rand_latent_point = latent_point(latent_dim, n_sample)
    x = gen.predict(rand_latent_point)
    y = zeros((n_sample, 1))
        
    return x, y
    
def evaluate_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = real_sample(n_samples, dataset)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = fake_sample(g_model, n_samples, latent_dim)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)
        
def save_plot(examples, epoch, n=10):
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()
    
def random_stack(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b
       
def train(gan, dis, epochs, batch_size, data, latent_dim=100):
    bat_per_epoch = int(data.shape[0]/batch_size)
    half_batch = int(batch_size/2)
        
    for i in range(epochs):
        for j in range(bat_per_epoch):
            real_x, real_y = real_sample(half_batch, data)
            fake_x, fake_y = fake_sample(gen, half_batch)
            x, y = vstack((real_x,fake_x)), vstack((real_y,fake_y))
            x, y = random_stack(x, y)
            dis_loss,_ = dis.train_on_batch(x, y)
            
            gan_x = latent_point(latent_dim, half_batch)
            gan_y = ones((half_batch, 1))
            gan_loss = gan.train_on_batch(gan_x, gan_y)
            if j%15==0:
                print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epoch, dis_loss, gan_loss))
                evaluate_performance(epoch=i, g_model=gen, d_model=dis, dataset=data, latent_dim=latent_dim)
                                        
        if (i+1) % 10 == 0:
            evaluate_performance(i, gen, dis, data, latent_dim)

In [ ]:
data = pd.read_csv('../input/digit-recognizer/train.csv')

data = data.drop(columns='label')
data = data.values
data = expand_dims(data, axis=-1)
data = data.astype('float32')
data = data/255

In [ ]:
dis = discriminator()
gen = generator(100)
gan = GAN(gen, dis)

In [ ]:
train(gan, dis, 100, 32, data)